In [1]:
# !pip install uavsar_pytools

In [5]:
from uavsar_pytools.UavsarScene import UavsarScene
from uavsar_pytools.UavsarImage import UavsarImage
from uavsar_pytools.uavsar_tools import create_netrc

In [9]:
directory_to_load = '~/Downloads/uavsar_ex/'
url = 'https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/lowman_05208_21019-019_21021-007_0006d_s01_L090_01_int_grd.zip'
scene = UavsarScene(url = url, work_dir = directory_to_load)

In [10]:
scene.url_to_tiffs()